# ND-Range Implementation for Matrix Multiplication

### Learning Objectives

- Understand how ND-range improves parallelism over the basic implementation.
- Able to explain why limiting global memory access can be advantageous.
- Explain the differences between work-groups and work-items.


## ND-Range Kernels

Our previous implementation of a naive parallel kernel did not allow for performance optimizations at a hardware level. In these next two kernels we will utilize ND-Range kernels as a way to expresses parallelism enabling low level performance tuning by providing access to both global and local memory and mapping executions to compute units on hardware. The entire iteration space is divided into smaller groups called work-groups, work-items are organized into these work-groups and are scheduled on a single compute unit on the hardware.  Workgroup size must divide the entire ND-range size exactly in each dimension.  These sizes can all vary by hardware platform and by using the device queries below a developer can identify what is possible.  The workload must be considered to find the best mix of these values.

```cpp
auto maxWG = dev.get_info<info::device::max_work_group_size>();
```

The grouping of kernel executions into work-groups allows control of resource usage and load balance work distribution. The functionality of nd_range kernels is exposed via nd_range and nd_item classes. nd_range class represents a grouped execution range using global execution range and the local execution range of each work-group. nd_item class represents an individual instance of a kernel function and allows you to query for work-group range and index.

<img src="Assets/workgroup.PNG">



## Matrix Multiplication with ND-Range Kernel

In the following two examples below we demonstrate using ND-Range Kernel to compute matrix multiplication. The work-group size depends on the accelerator hardware capability, so we set this size using command-line argument. Some hardware require the matrix size to divide equally by the work-group size, we will use work-group size of 16x16 (256) by default which works for all the accelerator hardware we will be using to test, we will eventually use different work-group sizes to see how it impacts the performance.

```cpp
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            for (int k = 0; k < N; k++) {
                C[i*N+j] += A[i*N+k] * B[k*N+j];
            }
        });
```

<img src="Assets/mm_wg.PNG">

The following SYCL code below shows an ND-range kernel: Inspect code, there are no modifications necessary:

1. Run the cell in the __Select Offload Device__ section to choose a target device to run the code on.
2. Inspect the following code cell and click __Run__ ▶ to save the code to a file.
3. Next, run the cell in the __Build and Run__ section to compile and execute the code.

#### Select Offload Device

In [ ]:
run accelerator.py

In [ ]:
%%writefile lab/mm_dpcpp_ndrange.cpp
//==============================================================
// Matrix Multiplication: SYCL ND-range
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <CL/sycl.hpp>

using namespace sycl;

void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M) {
    std::cout << "Configuration         : MATRIX_SIZE= " << N << "x" << N << " | WORK_GROUP_SIZE= " << M << "x" << M << "\n";
    
    //# Create buffers for matrices
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    auto e = q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        auto A = a.get_access<access::mode::read>(h);
        auto B = b.get_access<access::mode::read>(h);
        auto C = c.get_access<access::mode::write>(h);

        //# Define size for ND-Range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(M,M);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            for (int k = 0; k < N; k++) {
                C[i*N+j] += A[i*N+k] * B[k*N+j];
            }
        });
    });
    c.get_access<access::mode::read>();
    
    //# print kernel compute duration from event profiling
    auto kernel_duration = (e.get_profiling_info<info::event_profiling::command_end>() - e.get_profiling_info<info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time : " << kernel_duration / 1e+9 << " seconds\n";
}

#### Build and Run
Select the cell below and click __Run__ ▶ to compile and execute the code on selected device:

In [ ]:
! chmod 755 q; chmod 755 run_mm_ndrange.sh; if [ -x "$(command -v qsub)" ]; then ./q run_mm_ndrange.sh "{device.value}"; else ./run_mm_ndrange.sh; fi

### Roofline Report

Execute the following line to display the roofline results 


In [ ]:
run display_data/mm_ndrange_roofline.py

### VTune™ Profiler Summary

Execute the following line to display the VTune results.

In [ ]:
run display_data/mm_ndrange_vtune.py

## Matrix Multiplication with ND-Range using Private Memory

The example below shows writing the intermediate results within the loop into a variable defined inside the kernel which translates into a register in the accelerator hardware.  This minimizes the number of global memory writes and enhances performance as it only writes the result back once per work-group.  

```cpp
float temp = 0.f;
for (int k = 0; k < N; k++) {
    temp += A[i*N+k] * B[k*N+j];
}
C[i*N+j] = c;
```

The SYCL code below shows an ND-range kernel using private memory: Inspect code, there are no modifications necessary:

1. Run the cell in the __Select Offload Device__ section to choose a target device to run the code on.
2. Inspect the following code cell and click __Run__ ▶ to save the code to a file.
3. Next, run the cell in the __Build and Run__ section to compile and execute the code.

#### Select Offload Device

In [ ]:
run accelerator.py

In [ ]:
%%writefile lab/mm_dpcpp_ndrange_var.cpp
//==============================================================
// Matrix Multiplication: SYCL ND-range private mem
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <CL/sycl.hpp>

using namespace sycl;

void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M) {
    std::cout << "Configuration         : MATRIX_SIZE= " << N << "x" << N << " | WORK_GROUP_SIZE= " << M << "x" << M << "\n";

    //# Create buffers for matrices
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    auto e = q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        auto A = a.get_access<access::mode::read>(h);
        auto B = b.get_access<access::mode::read>(h);
        auto C = c.get_access<access::mode::write>(h);

        //# Define size for ND-Range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(M,M);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            //# Use private mem to store intermediate result
            float temp = 0.f;
            for (int k = 0; k < N; k++) {
                temp += A[i*N+k] * B[k*N+j];
            }
            C[i*N+j] = temp;
        });
    });
    c.get_access<access::mode::read>();
    
    //# print kernel compute duration from event profiling
    auto kernel_duration = (e.get_profiling_info<info::event_profiling::command_end>() - e.get_profiling_info<info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time : " << kernel_duration / 1e+9 << " seconds\n";
}

#### Build and Run
Select the cell below and click __Run__ ▶ to compile and execute the code on selected device:

In [ ]:
! chmod 755 q; chmod 755 run_mm_ndrange_var.sh; if [ -x "$(command -v qsub)" ]; then ./q run_mm_ndrange_var.sh; else ./run_mm_ndrange_var.sh; fi

Did you see identical times?  The default matrix size is 1024x1024 for speed across all implementations.  Compare the VTune data to see the results across larger matrix operations.

### Roofline Report

Execute the following line to display the roofline results 


In [ ]:
run display_data/mm_ndrange_var_roofline.py

### VTune™ Profiler Summary

Execute the following line to display the VTune results.

In [ ]:
run display_data/mm_ndrange_var_vtune.py

The size of the matrix and workgroup are all having an impact on the performance of the kernel across the platforms.  We see improvement across the hardware but we can see that work scheduling could be better optimized to take advantage of the wider number of EU's and CPU's available on DG1 and ATS.  We can see that the variable result copied to the device memory is improving performance with it being more noticeable the larger the array size.

## Analysis

Comparing the execution times for ND-Range SYCL implementation and ND-Range using private memory implementation for various matrix sizes, we can see that ND-Range with private memory implementation performs better. The graph below shows execution times on various hardware for matrix sizes 1024x1024, 5120x5120 and 10240x10240.

<img src=Assets/ppp_ndrange_graph.PNG>


### Summary

In this module we enhanced our basic parallel kernel by implementing two types of ND-Range kernels.  The first kernel relied on global memory and the second kernel used an intermediate variable to store results reducing the number of writes to global memory.  We seeing an increase in flops across all the platforms as compared to the basic implementation.  In the next module we will build on range kernels.

- Notices

Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure.
Your costs and results may vary.
© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.
